In [1]:
!pip install pandas as pd

ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as


In [2]:
import pandas as pd 
df = pd.read_csv("C:\\Users\\aryan\\Downloads\\synthetic_logs.csv")
df.head()

,timestamp,source,log_message,target_label,complexity
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert


In [3]:
df['source'].unique()


array(['ModernCRM', 'AnalyticsEngine', 'ModernHR', 'BillingSystem',
       'ThirdPartyAPI', 'LegacyCRM'], dtype=object)

In [4]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2410 entries, 0 to 2409
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   timestamp     2410 non-null   object
 1   source        2410 non-null   object
 2   log_message   2410 non-null   object
 3   target_label  2410 non-null   object
 4   complexity    2410 non-null   object
dtypes: object(5)
memory usage: 94.3+ KB


In [8]:
df.describe()


,timestamp,source,log_message,target_label,complexity
count,2410,2410,2410,2410,2410
unique,2407,6,2265,9,3
top,4/28/2025 20:42,ThirdPartyAPI,Backup completed successfully.,HTTP Status,bert
freq,2,496,60,1017,1903


In [10]:
!pip install sentence-transformers scikit-learn


   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ------------------- -------------------- 5.5/11.1 MB 25.7 MB/s eta 0:00:01
   --------------------------------- ------ 9.4/11.1 MB 22.5 MB/s eta 0:00:01
   ---------------------------------------- 11.1/11.1 MB 21.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/41.2 MB ? eta -:--:--
   ---- ----------------------------------- 4.7/41.2 MB 23.7 MB/s eta 0:00:02
   --------- ------------------------------ 9.4/41.2 MB 23.5 MB/s eta 0:00:02
   -------------- ------------------------- 14.7/41.2 MB 24.3 MB/s eta 0:00:02
   ------------------ --------------------- 19.4/41.2 MB 23.5 MB/s eta 0:00:01
   ---------------------- ----------------- 23.6/41.2 MB 23.3 MB/s eta 0:00:01
   --------------------------- ------------ 28.6/41.2 MB 22.9 MB/s eta 0:00:01
   ------------------------------- -------- 32.8/41.2 MB 23.1 MB/s eta 0:00:01


ERROR: Could not install packages due to an OSError: [Errno 28] No space left on device



In [5]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(df['log_message'].to_list())

# Step 2: Apply DBSCAN clustering
clustering = DBSCAN(eps=0.2, min_samples=1, metric='cosine')  # You can tune eps
labels = clustering.fit_predict(embeddings)
labels[:2]


array([0, 1], dtype=int64)

In [6]:
df['clusters'] = labels

In [7]:
df.head()

,timestamp,source,log_message,target_label,complexity,clusters
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert,0
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert,2
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert,0
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert,0


In [8]:
d1 = df.query('clusters == 1')
d1.head()

,timestamp,source,log_message,target_label,complexity,clusters
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1
10,8/9/2025 18:58,ModernCRM,Email server encountered a sending fault,Error,bert,1
217,1/22/2025 5:45,BillingSystem,Mail service encountered a delivery glitch,Error,bert,1
248,5/2/2025 23:04,ModernHR,Service disruption caused by email sending error,Critical Error,bert,1
265,3/30/2025 23:53,ModernCRM,Email system had a problem sending emails,Error,bert,1


In [14]:
cluster_count = df['clusters'].value_counts()
bigc = cluster_count[cluster_count>10].index
for c in bigc:
    print(f"cluster: {c}")
    print(df[df['clusters'] == c]['log_message'].head(5).to_string(index= False))
    
    print()
    

cluster: 0
nova.osapi_compute.wsgi.server [req-b9718cd8-f6...
nova.osapi_compute.wsgi.server [req-4895c258-b2...
nova.osapi_compute.wsgi.server [req-ee8bc8ba-92...
nova.osapi_compute.wsgi.server [req-f0bffbc3-5a...
nova.osapi_compute.wsgi.server [req-2bf7cfee-a2...

cluster: 5
nova.compute.claims [req-a07ac654-8e81-416d-bfb...
nova.compute.claims [req-d6986b54-3735-4a42-907...
nova.compute.claims [req-72b4858f-049e-49e1-b31...
nova.compute.claims [req-5c8f52bd-8e3c-41f0-95a...
nova.compute.claims [req-d38f479d-9bb9-4276-968...

cluster: 11
User User685 logged out.
 User User395 logged in.
 User User225 logged in.
User User494 logged out.
 User User900 logged in.

cluster: 13
Backup started at 2025-05-14 07:06:55.
Backup started at 2025-02-15 20:00:19.
  Backup ended at 2025-08-08 13:06:23.
Backup started at 2025-11-14 08:27:43.
Backup started at 2025-12-09 10:19:11.

cluster: 7
Multiple bad login attempts detected on user 85...
Multiple login failures occurred on user 9052 a...
  User 

In [29]:
import re
def classify_regex(log_m):
    regex_patterns = {
            # User Actions
            r"User User\d+ logged (in|out)\.": "User Action",
            r"Account with ID \d+ created by User\d+\.": "User Action",
        
            # System Notifications
            r"Backup (started|ended) at .*": "System Notification",
            r"Backup completed successfully\.": "System Notification",
            r"System updated to version .*": "System Notification",
            r"File .* uploaded successfully by user .*": "System Notification",
            r"Disk cleanup completed successfully\.": "System Notification",
            r"System reboot initiated by user User\d+\.": "System Notification",
        
            # Authentication/Access Issues
            r"Multiple (bad login attempts|login failures).*": "Security Alert",
            r"User \d+ made multiple incorrect login attempts": "Security Alert",
            r"Unauthorized login attempt on protected account.*": "Security Alert",
            r"Access denied.*account.*": "Security Alert",
            r"Invalid login attempt.*secured account.*": "Security Alert",
        
        
            # Input / Format Errors
            r"Module X .* (invalid data format|input format mismatch|error).*": "System Error",
        
            # Health Check / SSL Issues
            r"Service health check .* (failed|was not successful).*": "Security Alert",
            r"Invalid SSL certificate resulted in a failed service check.*": "Security Alert",
        
            # Anomalies / Suspicious Activity
            r"(Abnormal|suspicious|anomalous) activity .* server.*": "Security Alert"
        }
    for pattern, label in regex_patterns.items():
        if re.search(pattern, log_m, re.IGNORECASE):
            return label
    return None
print(classify_regex("Backup started at 2025-05-14 07:06:55."))

System Notification


In [30]:
df['regex_label'] = df['log_message'].apply(classify_regex)
df.head(5)

,timestamp,source,log_message,target_label,complexity,clusters,regex_label
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert,0,None
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1,None
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert,2,None
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert,0,None
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert,0,None


In [31]:
d2 = df.query('regex_label == "Security Alert"')
d2.head()


,timestamp,source,log_message,target_label,complexity,clusters,regex_label
13,8/4/2025 19:57,ThirdPartyAPI,Multiple bad login attempts detected on user 8...,Security Alert,bert,7,Security Alert
53,7/29/2025 0:17,ModernHR,Multiple login failures occurred on user 9052 ...,Security Alert,bert,7,Security Alert
67,5/12/2025 2:50,ModernHR,Unauthorized login attempt on protected accoun...,Security Alert,bert,17,Security Alert
72,2/3/2025 5:35,AnalyticsEngine,User 7153 made multiple incorrect login attempts,Security Alert,bert,7,Security Alert
74,5/9/2025 21:18,BillingSystem,User 8300 made multiple incorrect login attempts,Security Alert,bert,7,Security Alert


In [32]:
print(df.shape)
print(df[df['regex_label'].notnull()].shape)

(2410, 7)
(532, 7)


In [41]:
df_nregex = df[df['regex_label'].isnull()]
df_nregex.head()



,timestamp,source,log_message,target_label,complexity,clusters,regex_label
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert,0,None
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1,None
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert,2,None
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert,0,None
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert,0,None


In [43]:
l_count = df_nregex['target_label'].value_counts()
smalll = l_count[l_count < 5].index
for l in smalll:
    print(f"target label with insufficient training samples: {l}")




target label with insufficient training samples: Workflow Error
target label with insufficient training samples: Deprecation Warning


In [35]:
df_lr = df_nregex[~df_nregex['source'].isin(['Workflow Error', 'Deprecation Warning'])]


In [36]:
df_lr.head()

,timestamp,source,log_message,target_label,complexity,clusters,regex_label
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert,0,None
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1,None
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert,2,None
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert,0,None
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert,0,None


In [45]:
df_llm = df_nregex[df_nregex['target_label'].isin(['Workflow Error', 'Deprecation Warning'])]
df_llm.head(10)

,timestamp,source,log_message,target_label,complexity,clusters,regex_label
60,2025-10-06 16:55:23,LegacyCRM,Lead conversion failed for prospect ID 7842 du...,Workflow Error,llm,24,None
255,2025-05-03 16:55:35,LegacyCRM,API endpoint 'getCustomerDetails' is deprecate...,Deprecation Warning,llm,48,None
377,2025-06-24 12:16:29,LegacyCRM,Customer follow-up process for lead ID 5621 fa...,Workflow Error,llm,62,None
1325,2025-04-17 07:33:44,LegacyCRM,Escalation rule execution failed for ticket ID...,Workflow Error,llm,105,None
1734,2025-04-30 07:47:30,LegacyCRM,The 'ExportToCSV' feature is outdated. Please ...,Deprecation Warning,llm,118,None
1826,2025-01-23 10:33:36,LegacyCRM,Support for legacy authentication methods will...,Deprecation Warning,llm,122,None
2217,2025-05-12 09:46:54,LegacyCRM,Task assignment for TeamID 3425 could not comp...,Workflow Error,llm,133,None


In [46]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

model = SentenceTransformer('all-MiniLM-L6-v2')
f_embeddings = model.encode(df_lr['log_message'].to_list())

X = f_embeddings


y = df_lr['target_label']


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)


y_pred = clf.predict(X_test)


print(classification_report(y_test, y_pred))


                     precision    recall  f1-score   support

     Critical Error       0.97      1.00      0.98        32
Deprecation Warning       0.00      0.00      0.00         1
              Error       0.97      0.97      0.97        34
        HTTP Status       1.00      1.00      1.00       204
     Resource Usage       1.00      1.00      1.00        35
     Security Alert       0.99      1.00      0.99        69
     Workflow Error       0.00      0.00      0.00         1

           accuracy                           0.99       376
          macro avg       0.70      0.71      0.71       376
       weighted avg       0.99      0.99      0.99       376



E:\USER\aryan\media\envs\GM\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
E:\USER\aryan\media\envs\GM\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
E:\USER\aryan\media\envs\GM\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [48]:
import joblib
joblib.dump(clf,'log_classifier.joblib')

['log_classifier.joblib']

In [ ]:
import joblib